In [2]:
import sqlite3
import csv

# Chemin du fichier CSV
csv_file_path = "raw_data/sales.csv"

# Connexion à la base de données SQLite
conn = sqlite3.connect("sales.db")
cursor = conn.cursor()

# Création de la table avec des types adaptés
cursor.execute('''
CREATE TABLE IF NOT EXISTS sales (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    date TEXT,           -- Format YYYY-MM-DD
    datetime TEXT,       -- Format YYYY-MM-DD HH:MM:SS.sss
    cash_type TEXT,
    card TEXT,
    money REAL,
    coffee_name TEXT
)
''')

# Lecture et insertion des données du fichier CSV
with open(csv_file_path, mode='r', encoding='utf-8') as file:
    reader = csv.reader(file)
    
    # Ignorer les en-têtes
    headers = next(reader)
    
    # Insérer les lignes dans la table SQLite
    for row in reader:
        cursor.execute('''
        INSERT INTO sales (date, datetime, cash_type, card, money, coffee_name)
        VALUES (?, ?, ?, ?, ?, ?)
        ''', row)

# Sauvegarder les changements et fermer la connexion
conn.commit()
conn.close()

print("Les données ont été importées dans la base de données 'sales.db'.")

Les données ont été importées dans la base de données 'sales.db'.


In [3]:
import sqlite3
import pandas as pd

# Connexion à la base de données SQLite
conn = sqlite3.connect("sales.db")

# Charger les données de la table 'sales' dans un DataFrame Pandas
query = "SELECT * FROM sales"
data = pd.read_sql_query(query, conn)

# Conversion des colonnes 'date' et 'datetime' en type datetime
data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')
data['datetime'] = pd.to_datetime(data['datetime'], format='%Y-%m-%d %H:%M:%S.%f')

# Afficher des informations générales sur les données
print("Informations générales sur les données :")
print(data.info())

# Afficher les 5 premières lignes des données
print("\nPremières lignes des données :")
print(data.head())

# Fermeture de la connexion
conn.close()

Informations générales sur les données :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15738 entries, 0 to 15737
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   id           15738 non-null  int64         
 1   date         15738 non-null  datetime64[ns]
 2   datetime     15738 non-null  datetime64[ns]
 3   cash_type    15738 non-null  object        
 4   card         15738 non-null  object        
 5   money        15738 non-null  float64       
 6   coffee_name  15738 non-null  object        
dtypes: datetime64[ns](2), float64(1), int64(1), object(3)
memory usage: 860.8+ KB
None

Premières lignes des données :
   id       date                datetime cash_type                 card  \
0   1 2024-03-01 2024-03-01 10:15:50.520      card  ANON-0000-0000-0001   
1   2 2024-03-01 2024-03-01 12:19:22.539      card  ANON-0000-0000-0002   
2   3 2024-03-01 2024-03-01 12:20:18.089      card  ANON-0000-000

In [4]:
import sqlite3
import pandas as pd
from datetime import datetime

# Connexion à la base de données SQLite
conn = sqlite3.connect("sales.db")

# Charger les données de la table 'sales' dans un DataFrame Pandas
query = "SELECT * FROM sales"
data = pd.read_sql_query(query, conn)

# Conversion des colonnes 'date' et 'datetime' en type datetime
data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')
data['datetime'] = pd.to_datetime(data['datetime'], format='%Y-%m-%d %H:%M:%S.%f')

# Trouver la dernière date dans les données
derniere_date = data['date'].max()
dernier_mois = derniere_date.month
derniere_annee = derniere_date.year

# Déterminer le mois et l'année du mois précédent
if dernier_mois == 1:  # Si janvier, alors le mois précédent est décembre de l'année précédente
    mois_precedent = 12
    annee_precedente = derniere_annee - 1
else:
    mois_precedent = dernier_mois - 1
    annee_precedente = derniere_annee

# Filtrer les données pour le dernier mois
data_dernier_mois = data[(data['date'].dt.month == dernier_mois) & (data['date'].dt.year == derniere_annee)]

# Filtrer les données pour le mois précédent
data_mois_precedent = data[(data['date'].dt.month == mois_precedent) & (data['date'].dt.year == annee_precedente)]

# Calcul des KPIs pour le dernier mois
total_ca_dernier_mois = round(data_dernier_mois['money'].sum(), 0)
nombre_transactions_dernier_mois = len(data_dernier_mois)
moyenne_ca_dernier_mois = round(data_dernier_mois['money'].mean(), 0)
produit_populaire_dernier_mois = data_dernier_mois['coffee_name'].mode()[0]

# Calcul des KPIs pour le mois précédent
total_ca_mois_precedent = round(data_mois_precedent['money'].sum(), 0)
nombre_transactions_mois_precedent = len(data_mois_precedent)
moyenne_ca_mois_precedent = round(data_mois_precedent['money'].mean(), 0)
produit_populaire_mois_precedent = data_mois_precedent['coffee_name'].mode()[0]

# Calcul des pourcentages de variation
pourcentage_variation_ca = ((total_ca_dernier_mois - total_ca_mois_precedent) / total_ca_mois_precedent) * 100 if total_ca_mois_precedent != 0 else 0
pourcentage_variation_transactions = ((nombre_transactions_dernier_mois - nombre_transactions_mois_precedent) / nombre_transactions_mois_precedent) * 100 if nombre_transactions_mois_precedent != 0 else 0
pourcentage_variation_moyenne_ca = ((moyenne_ca_dernier_mois - moyenne_ca_mois_precedent) / moyenne_ca_mois_precedent) * 100 if moyenne_ca_mois_precedent != 0 else 0

# Stocker les résultats dans une variable texte
resultats_kpis = f"""
KPIs pour le dernier mois :
- Total du chiffre d'affaires : {total_ca_dernier_mois}
- Nombre de transactions : {nombre_transactions_dernier_mois}
- Moyenne du chiffre d'affaires par transaction : {moyenne_ca_dernier_mois}
- Produit le plus populaire : {produit_populaire_dernier_mois}

KPIs pour le mois précédent :
- Total du chiffre d'affaires : {total_ca_mois_precedent}
- Nombre de transactions : {nombre_transactions_mois_precedent}
- Moyenne du chiffre d'affaires par transaction : {moyenne_ca_mois_precedent}
- Produit le plus populaire : {produit_populaire_mois_precedent}

Tendances en pourcentage par rapport au mois précédent :
- Variation du chiffre d'affaires : {pourcentage_variation_ca:.2f}%
- Variation des transactions : {pourcentage_variation_transactions:.2f}%
- Variation de la moyenne du chiffre d'affaires : {pourcentage_variation_moyenne_ca:.2f}%
"""

# Afficher les résultats
print(resultats_kpis)

# Fermeture de la connexion
conn.close()



KPIs pour le dernier mois :
- Total du chiffre d'affaires : 36318.0
- Nombre de transactions : 1134
- Moyenne du chiffre d'affaires par transaction : 32.0
- Produit le plus populaire : Americano with Milk

KPIs pour le mois précédent :
- Total du chiffre d'affaires : 51543.0
- Nombre de transactions : 1554
- Moyenne du chiffre d'affaires par transaction : 33.0
- Produit le plus populaire : Latte

Tendances en pourcentage par rapport au mois précédent :
- Variation du chiffre d'affaires : -29.54%
- Variation des transactions : -27.03%
- Variation de la moyenne du chiffre d'affaires : -3.03%



In [5]:
import json
from openai import OpenAI

# Charger la configuration
with open('config.json', 'r') as f:
    config = json.load(f)

# Initialiser le client
client = OpenAI(
    base_url=config["base_url"],
    api_key=config["api_key"]
)

# Prompt simplifié
question = f"""
Rédige un message d'équipe motivant basé sur ces KPIs :
{resultats_kpis}

Ton rôle : Manager encourageant
Ton style : Professionnel et humain
Format : Message court et structuré
Ton objectif : Analyser, expliquer et motiver
"""

# Utilisation
response = client.chat.completions.create(
    extra_headers=config["headers"],
    model="deepseek/deepseek-r1:free",
    messages=[{
        "role": "user",
        "content": question
    }]
)

print(response.choices[0].message.content)

**Message à l'équipe**  
🌟 **Chers tous,**  

**Analyse des résultats** :  
Ce mois-ci, nous avons généré **36 318€** de CA (vs 51 543€ le mois dernier) avec **1 134 transactions** (vs 1 554). L’Americano with Milk a été notre star produit, remplaçant le Latte en tête des ventes !  

**Contexte** :  
La baisse du CA (-29,5%) et des transactions (-27%) reflète un contexte exigeant, mais souligne aussi des opportunités. La légère baisse du panier moyen (-3%) montre que chaque client compte plus que jamais.  

**Leviers d'action** :  
1️⃣ **Capitalisons sur l’Americano with Milk** : Promouvons des combinaisons gagnantes (viennoiseries, snacks) pour augmenter le panier moyen.  
2️⃣ **Ciblons la fidélité** : Relançons les clients peu actifs via des offres personnalisées.  
3️⃣ **Restons agiles** : Observons les tendances et adaptons nos promotions en temps réel.  

**Motivation** :  
Les défis sont des tremplins pour innover ! Chaque sourire, chaque recommandation, chaque détail fait la dif

In [6]:
analyse_reunion = """
**Transcription Brainstorming - 15/11 - Café "Les Grains d'Or"**

**Participants** : 
- Sophie (Manager)
- Marc (Barista chef)
- Thomas (Responsable appro)
- Laura (Marketing)

**Sophie** : "On a vu -18% de CA ce mois-ci, c'est sérieux. Marc, tes observations en salle ?"

**Marc** : "Clairement moins de monde entre 14h-16h. Les étudiants venaient avant avec leurs ordis, maintenant je les vois au nouveau Bean Brothers. Leur wifi est gratuit en plus..."

**Laura** : "Justement, j'ai checké leurs prix : leur latte est à 4,20€ contre nos 4,80€. Et ils font des cookies offerts après 15h."

**Thomas** : "On a eu un souci technique aussi - la machine à lait a fui du 15 au 18 octobre. J'ai dû servir des cafés noirs à des clients qui voulaient des lattes..."

**Marc** : "Oui, et 3 clients m'ont dit qu'ils trouvaient nos nouvelles tasses trop petites. La taille 'grand' passe de 350 à 300ml."

**Sophie** : "Thomas, des insights sur les appros ?"

**Thomas** : "Le prix du lait bio a augmenté de 12% ce mois-ci. Et notre fournisseur de pâtisseries a eu 5 jours de retard."

**Laura** : "Par contre, bonne nouvelle : nos posts Instagram sur le latte art ont généré +320 leads. Mais ça convertit pas assez..."

**Échanges clés** :
1. "Bean Brothers nous pique la clientèle étudiante avec des prix agressifs" → Marc
2. "La panne machine + taille des tasses = double peine sur les cafés crèmeux" → Thomas
3. "On communique sur la qualité mais sans preuve tangible" → Laura

**Idées spontanées** :
- "Et si on faisait des ateliers latte art le mercredi après-midi ?" → Marc
- "Proposer un tarif étudiant avec carte U ?" → Laura
- "Recalibrer nos tailles de tasses pour garder la marge mais paraître plus généreux" → Thomas
"""

In [7]:
import json
from openai import OpenAI

# Charger la configuration
with open('config.json', 'r') as f:
    config = json.load(f)

# Initialiser le client
client = OpenAI(
    base_url=config["base_url"],
    api_key=config["api_key"]
)


# Construction du prompt RAG
prompt_analyse = f"""
Analysez ces résultats KPI en tenant compte des insights de notre dernière réunion d'analyse.
Fournissez une synthèse actionable pour l'équipe.

CONTEXTE (extrait de la réunion du 15/11) :
{analyse_reunion}

DONNÉES ACTUELLES À ANALYSER :
{resultats_kpis}

Consignes :
1. Faites des liens concrets entre les tendances actuelles et nos conclusions passées
2. Ton style : synthétique, orienté solutions, avec données clés en évidence
"""

# Requête au modèle
response = client.chat.completions.create(
    extra_headers=config["headers"],
    model="deepseek/deepseek-r1:free",
    messages=[{
        "role": "user",
        "content": prompt_analyse
    }]
)

print("ANALYSE CONTEXTUALISÉE DES KPIs\n")
print(response.choices[0].message.content)

ANALYSE CONTEXTUALISÉE DES KPIs

**Synthèse Actionnable - Café "Les Grains d'Or"**  
*Liens entre données et insights :*  

1. **Chute de 29.5% du CA** → Causée par :  
   - **27% moins de transactions** : Migration des étudiants vers Bean Brothers (wifi gratuit, prix bas).  
   - **Problème machine à lait** (15-18 oct) → Clients refusés pour lattes → **Americano with Milk** nouveau best-seller.  

2. **Baisse de 3% du panier moyen** → Associée à :  
   - Réduction taille des tasses (300ml vs 350ml) → Perçue comme moins généreuse.  
   - Offres promotionnelles concurrentielles (ex: cookie offert à 15h chez Bean Brothers).  

3. **Opportunité non exploitée** :  
   - **+320 leads Instagram** non convertis → Manque d'incitation claire (ex: promo ciblée).  

---

**Actions Prioritaires (Owner) :**  

**🟢 Recapture Étudiants (Marc/Laura)**  
- Tester **tarif réduit étudiant (4€ le latte) 14h-16h** avec présentation de carte étudiante.  
- **WiFi gratuit** *uniquement* sur achat (minimum 1 